In [3]:
# Import necessary TensorFlow modules
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

## Importing TensorFlow Modules and Data Preprocessing
This code imports essential TensorFlow modules and performs data preprocessing for training, validation, and testing datasets. The datasets are loaded from directories using tf.keras.preprocessing.image_dataset_from_directory. The training dataset is split into training and validation subsets with a validation split of 20%. The image_size parameter resizes all images to 224x224 pixels, and the batch_size parameter sets the number of samples per batch to 32. A seed is used for reproducibility to ensure consistent data splitting across runs. This setup prepares the datasets for training a machine learning model, ensuring that the images are uniformly sized and batched.

In [12]:

# Import necessary TensorFlow modules
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

# Data preprocessing
# Load the training dataset from the directory with a validation split
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/Users/shahadaleissa/CV-labs/Data/archive/train',
    validation_split=0.2,
    subset="training",
    seed=123,  # Seed for reproducibility
    image_size=(224, 224),  # All images will be resized to 224x224
    batch_size=32  # Number of samples per batch
)

# Load the validation dataset from the same directory
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/Users/shahadaleissa/CV-labs/Data/archive/train',
    validation_split=0.2,
    subset="validation",
    seed=123,  # Using the same seed for consistent data splitting
    image_size=(224, 224),
    batch_size=32
)

# Load the test dataset from a different directory
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/Users/shahadaleissa/CV-labs/Data/archive/images',
    seed=123,  # Seed for reproducibility
    image_size=(224, 224),
    batch_size=32
)

Found 2746 files belonging to 5 classes.
Using 2197 files for training.
Found 2746 files belonging to 5 classes.
Using 549 files for validation.
Found 924 files belonging to 1 classes.


## Model Setup Using MobileNetV2
This code sets up a neural network model for image classification using a pre-trained MobileNetV2 model from TensorFlow Hub as a feature extractor. The module_url specifies the location of the pre-trained MobileNetV2 model. A tf.keras.Sequential model is constructed with two layers: a hub.KerasLayer that loads the pre-trained model for feature extraction and is set to non-trainable, followed by a dense output layer with a softmax activation function to classify the images into 5 classes. This setup leverages transfer learning to efficiently train a model for a specific task using pre-trained features.



In [20]:

# Model setup
# Load a pre-trained MobileNetV2 model from TensorFlow Hub as a feature extractor
module_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5"
model = tf.keras.Sequential([
    hub.KerasLayer(module_url, trainable=False),  # Feature extraction layer
    layers.Dense(5, activation='softmax')  # Output layer with softmax for 5 classes
])


## Configuring the Dataset for Performance
This code optimizes the training and validation datasets for performance using TensorFlow's data pipeline utilities. The AUTOTUNE parameter enables dynamic tuning of the data loading process. The cache() method caches the dataset in memory to avoid the overhead of re-reading the data from disk for each epoch. The prefetch(buffer_size=AUTOTUNE) method allows data to be preloaded in the background while the model is training, ensuring that data is always available when needed and reducing idle time. These optimizations help to speed up the training process by efficiently managing data loading and preprocessing.

In [13]:
# Configure the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)  # Optimize loading data
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [21]:
# Compile the model with optimization settings
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [22]:
# Train the model
epochs = 15  # Adjust the number of epochs as needed
history = model.fit(train_dataset, epochs=epochs, validation_data=validation_dataset)


Epoch 1/10


69/69 [==============================] - 24s 305ms/step - loss: 1.4512 - accuracy: 0.4133 - val_loss: 1.3037 - val_accuracy: 0.4499
Epoch 2/10
69/69 [==============================] - 11s 162ms/step - loss: 1.2104 - accuracy: 0.5448 - val_loss: 1.2001 - val_accuracy: 0.5100
Epoch 3/10
69/69 [==============================] - 11s 160ms/step - loss: 1.1115 - accuracy: 0.5835 - val_loss: 1.1494 - val_accuracy: 0.5228
Epoch 4/10
69/69 [==============================] - 11s 159ms/step - loss: 1.0462 - accuracy: 0.6099 - val_loss: 1.1176 - val_accuracy: 0.5501
Epoch 5/10
69/69 [==============================] - 11s 158ms/step - loss: 0.9964 - accuracy: 0.6309 - val_loss: 1.0957 - val_accuracy: 0.5628
Epoch 6/10
69/69 [==============================] - 11s 158ms/step - loss: 0.9559 - accuracy: 0.6509 - val_loss: 1.0800 - val_accuracy: 0.5774
Epoch 7/10
69/69 [==============================] - 11s 159ms/step - loss: 0.9216 - accuracy: 0.6650 - val_loss: 1.0685 - val_accuracy: 0.5683
Epoch 8/10

In [19]:
# Evaluate the model on the test dataset
model.evaluate(test_dataset)

29/29 [==============================] - 2s 48ms/step - loss: 8.7687 - accuracy: 0.2673


[8.768701553344727, 0.2673160135746002]